In [1]:
from keras import Model, layers, Sequential, optimizers
from keras.layers import Dense, Flatten
from tensorflow.keras.layers.experimental.preprocessing import Resizing
import numpy as np

2023-11-28 12:57:04.129506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 12:57:04.223049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-28 12:57:04.223061: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-28 12:57:04.241868: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 12:57:04.869793: W tensorflow/stream_executor/platform/de

In [2]:
from tensorflow.keras import datasets

(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((50000, 32, 32, 3), (50000, 1)), ((10000, 32, 32, 3), (10000, 1)))

In [3]:
classes = len(np.unique(y_train))
classes

10

In [4]:
from tensorflow.keras.utils import to_categorical

# $CHALLENGIFY_BEGIN
y_train_cat = to_categorical(y_train, num_classes=classes)
y_test_cat = to_categorical(y_test, num_classes=classes)
# $CHALLENGIFY_END

In [5]:
y_train_cat.shape

(50000, 10)

In [6]:
model = Sequential()

model.add(Resizing(224, 224, interpolation='bilinear', input_shape=(None, None, 3)))

model.add(layers.Conv2D(32, (3, 3), activation='relu')) #, input_shape=(32, 32, 3)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(classes, activation='softmax'))

2023-11-28 12:57:07.138387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 12:57:07.138495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-28 12:57:07.138536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-11-28 12:57:07.138574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-11-28 12:57:07.138611: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [7]:
learning_rate_=0.0005

optimizer_ = optimizers.Adam(learning_rate=learning_rate_)

metrics_ = ["accuracy"]

model.compile(loss="categorical_crossentropy", optimizer=optimizer_, metrics=metrics_)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        3

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 2)

history = model.fit(X_train, 
                    y_train_cat,
                    validation_split = 0.3,
                    batch_size = 32,
                    epochs = 5,
                    callbacks = [es],
                    verbose = 1)

Epoch 1/5
1094/1094 [==============================] - 4843s 4s/step - loss: 3.5155 - accuracy: 0.1933 - val_loss: 2.2451 - val_accuracy: 0.1549
Epoch 2/5
1094/1094 [==============================] - 751s 687ms/step - loss: 2.0338 - accuracy: 0.2756 - val_loss: 2.0290 - val_accuracy: 0.2741
Epoch 3/5
 798/1094 [====================>.........] - ETA: 3:04 - loss: 1.8275 - accuracy: 0.3615

In [ ]:
res = model.evaluate(X_test, y_test_cat, verbose = 1 )
print(f'The accuracy on the test set is of {res[1]*100:.2f} %')

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
X_pred = X_test[0]

In [ ]:
model.predict(X_pred)